# Creating new dataset for finetuning...

The idea here is to make RAG for GPT-4 and local LLMs like LLaMA 2 or Mistral. An important step is getting context for our training data, so in this notebook we will be running retrieval on all the questions in our dataset and creating new prompts for each question to finetune on.

In [1]:
from text_retriever import TextRetriever
from datasets import load_dataset

dataset = load_dataset("msaad02/brockport-gpt-4-qa")['train'].to_pandas()

retriever = TextRetriever()

2024-01-03 13:21:12.527480: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-03 13:21:12.549631: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 13:21:12.989953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset['context'] = dataset['question'].apply(lambda x: retriever.retrieve(x))

In [4]:
dataset.to_csv("data_with_context.csv", index=False)